### **Loading Step**

In [1]:
import os
import getpass
from pathlib import Path
import warnings

warnings.filterwarnings("ignore")

from docling.document_converter import DocumentConverter

In [2]:
source = r"code_du_travail_senegal.pdf"

In [3]:
converter = DocumentConverter()
result = converter.convert(source)

In [8]:
document = result.document.export_to_markdown()
type(document)

str

In [10]:
# Markdown splitting
from langchain_text_splitters import MarkdownHeaderTextSplitter

document = result.document.export_to_markdown()
headers_to_split_on = [
    ("#", "Header 1"), ("##", "Header 2"),
    ("###", "Header 3"), ("####", "Header 4"),
    ("#####", "Header 5"), ("######", "Header 6"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on, 
                                               return_each_line=True)

md_header_splits = markdown_splitter.split_text(document)

In [11]:
len(md_header_splits)

847

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=30
)

splits = text_splitter.split_documents(md_header_splits)

In [13]:
len(splits)

1272

In [17]:
# Initialize the embedding model from Hugging Face
from langchain_huggingface import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-base")

In [18]:
from langchain_community.vectorstores import Chroma

# Create the vector store from the chunks and embedding model
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)

In [19]:
retriever = vectorstore.as_retriever(k=4)

In [20]:
from langchain.prompts import PromptTemplate

# Définir un prompt template pour le LLM
prompt = PromptTemplate(
    template="""You are an assistant for question-answering tasks.
    Use the following documents to answer the question.
    If you don't know the answer, just say that you don't know.
    Provide your answers in French.
    Use three sentences maximum and keep the answer concise:
    Question: {question}
    Documents: {documents}
    Answer:
    """,
    input_variables=["question", "documents"],
)

In [21]:
# Avec Gemini
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Entrez votre clé API Google:")

from langchain.chat_models import init_chat_model

llm_model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [22]:
# Créer une chaine combinant le modele d'invite et LLM
from langchain_core.output_parsers import StrOutputParser

rag_chain = prompt | llm_model | StrOutputParser()

In [23]:
# Définir la classe d'application RAG
class RAGApplication:
    def __init__(self, retriever, rag_chain):
        self.retriever = retriever
        self.rag_chain = rag_chain
    def run(self, question):
      # Récupérer les documents pertinents
        documents = self.retriever.invoke(question)
      # Extraire le contenu des documents récupérés
        doc_texts = "\\n".join([doc.page_content for doc in documents])
      # Obtenir la réponse à partir du modèle de langage
        answer = self.rag_chain.invoke({"question": question, "documents": doc_texts})
        return answer

In [27]:
# Initialiser l'application de RAG
rag_application = RAGApplication(retriever, rag_chain)
# Example d'usage
question = "Qu'est ce qu'une banane?"
answer = rag_application.run(question)
print("Question:", question)
print("Answer:", answer)

Question: Qu'est ce qu'une banane?
Answer: Je ne dispose pas des informations nécessaires pour répondre à cette question dans les documents fournis. Les documents ne contiennent aucune information sur ce qu'est une banane.


In [28]:
question = "Qu'est ce que le code du travail du senegal dit?"
answer = rag_application.run(question)
print("Question:", question)
print("Answer:", answer)

Question: Qu'est ce que le code du travail du senegal dit?
Answer: Le Code du travail du Sénégal stipule que tout contrat de travail conclu pour être exécuté sur son territoire est soumis à ses dispositions. Cela s'applique quels que soient le lieu de la conclusion du contrat et la résidence de l'une ou l'autre partie.


In [29]:
question = "Quelle est le SMIG au Sénégal?"
answer = rag_application.run(question)
print("Question:", question)
print("Answer:", answer)

Question: Quelle est le SMIG au Sénégal?
Answer: Je ne dispose pas de l'information sur le montant exact du SMIG au Sénégal dans les documents fournis. Les documents mentionnent le "salaire minimum interprofessionnel garanti" et des textes législatifs y afférents, mais pas sa valeur actuelle.


In [30]:
question = "Comment le Code du travail traite-t-il des contrats à durée déterminée (CDD) et indéterminée (CDI) ?"
answer = rag_application.run(question)
print("Question:", question)
print("Answer:", answer)

Question: Comment le Code du travail traite-t-il des contrats à durée déterminée (CDD) et indéterminée (CDI) ?
Answer: Le Code du travail considère le contrat à durée indéterminée (CDI) comme la forme par défaut si les conditions spécifiques d'autres types de contrats ne sont pas remplies. Le contrat à durée déterminée (CDD) doit obligatoirement être constaté par écrit, sinon il est présumé conclu pour une durée indéterminée. De plus, le CDD ne peut avoir pour objet de pourvoir durablement un emploi lié à l'activité normale et permanente de l'entreprise.


In [33]:
question = "Que dit l'Article L.5."
answer = rag_application.run(question)
print("Question:", question)
print("Answer:", answer)

Question: Que dit l'Article L.5.
Answer: Je ne dispose pas d'informations sur l'article L.5 dans les documents fournis.


In [37]:
question = "Quelles sont les acticles qui abordent les contrats à durée déterminée au Sénégal?"
answer = rag_application.run(question)
print("Question:", question)
print("Answer:", answer)

Question: Quelles sont les acticles qui abordent les contrats à durée déterminée au Sénégal?
Answer: Les articles L.57 à L.59 sont applicables aux contrats à durée déterminée.
